In [29]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
# audio_dataset.ipynb 에서 만든 데이터셋 불러오기

import datasets

minds = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/dataset_01")

In [31]:
# model-2 : 추가데이터
angry_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/angry_dataset_hugging")
happy_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/HAPPY_dataset_hugging")
neutral_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/NEUTRAL_dataset_hugging")

In [32]:
from datasets import Dataset, concatenate_datasets

combined_dataset = concatenate_datasets([angry_dataset, happy_dataset, neutral_dataset])
combined_dataset

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 3000
})

In [37]:
type(happy_dataset[0]['audio']['array'])

numpy.ndarray

In [38]:
type(minds[0]['audio']['array'])

list

In [39]:
from datasets import Audio

combined_dataset = combined_dataset.cast_column("audio", Audio(sampling_rate=16_000))

In [41]:
combined_dataset[0]['audio']

{'path': 'F_01_OISHI_S_10_ANGRY_1.wav',
 'array': array([ 1.36005406e-06, -3.75603179e-07,  6.17460501e-08, ...,
         4.02642536e-06, -3.93963228e-06,  7.52304641e-06]),
 'sampling_rate': 16000}

In [42]:
from datasets import Audio

minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [43]:
minds[0]['audio']

{'path': None,
 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         8.88178420e-12, -7.21911420e-12,  5.61328761e-12]),
 'sampling_rate': 16000}

In [8]:
neutral_dataset['intent_class'][:2]

[0, 0]

In [9]:
minds['intent_class'][:2]

[1, 1]

In [45]:
label2id = {'angry':'1','neutral':'0'}
id2label = {'1':'angry','0':'neutral'}
num_labels = len(id2label)

In [46]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [12]:
from datasets import Audio

minds = minds.cast_column("audio", Audio(sampling_rate=16_000))

In [13]:
minds

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 5501
})

In [14]:
combined_dataset

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 3000
})

In [15]:
minds2 = concatenate_datasets([minds, combined_dataset])

In [16]:
minds2

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 8501
})

In [17]:
minds2 = minds.train_test_split(test_size=0.2)
minds2

DatasetDict({
    train: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 4400
    })
    test: Dataset({
        features: ['audio', 'intent_class'],
        num_rows: 1101
    })
})

In [27]:
minds2['train'][0]['audio']

{'path': None,
 'array': array([0.01272583, 0.02197266, 0.01123047, ..., 0.        , 0.        ,
        0.        ]),
 'sampling_rate': 16000}

In [19]:
minds2["train"][0]

{'audio': {'path': None,
  'array': array([0.01272583, 0.02197266, 0.01123047, ..., 0.        , 0.        ,
         0.        ]),
  'sampling_rate': 16000},
 'intent_class': 0}

In [20]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [21]:
encoded_minds = minds2.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds = encoded_minds.rename_column("intent_class", "label")

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [22]:
import evaluate

accuracy = evaluate.load("accuracy")

In [23]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [24]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.bias', 'classifier.bias', 'classifier.weight', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
training_args = TrainingArguments(
    output_dir="my_voice_classification_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    push_to_hub=True,
    # no_cuda =True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch,Training Loss,Validation Loss,Accuracy
0,0.599600,0.549646,0.645777
2,0.343800,0.312054,0.889192
2,0.267000,0.276332,0.893733
4,0.250400,0.230192,0.910082
4,0.216100,0.214070,0.914623
6,0.225600,0.197055,0.922797
6,0.181000,0.194505,0.932788
8,0.160400,0.184090,0.931880
8,0.141600,0.192793,0.932788
10,0.132400,0.227256,0.914623


TrainOutput(global_step=680, training_loss=0.17248314496348885, metrics={'train_runtime': 1256.998, 'train_samples_per_second': 70.008, 'train_steps_per_second': 0.541, 'total_flos': 7.8744428066304e+17, 'train_loss': 0.17248314496348885, 'epoch': 19.71})

In [26]:
trainer.save_model("./model/model-result-3")